In [1]:
import io
import ogr
import shapely.wkt
import shapely.geometry
import urllib.request
import zipfile
import time
import boto3
from mpi4py import MPI

ModuleNotFoundError: No module named 'mpi4py'

In [2]:
BUCKET = 'landsat-pds'

In [3]:
lon = -123  # Vancouver lon deg E
lat = 49   # Vancouver lat deg N

test = []
test.append(( -62.91692, 46.10125))
test.append((9.55137, 2.94979))
test.append((86.14700, 31.66302))
test.append((9.28831, 12.13683))

In [4]:
# source: https://a301_web.eoas.ubc.ca/week9/landsat_wrs.html
def checkPoint(feature, point, mode):
    '''
    Checks to see whether a lat/lon point falls within a
    particular WRS sector (feature)
    '''
    #Get geometry from feature
    geom = feature.GetGeometryRef() 
    
    #Import geometry into shapely to easily work with our point
    shape = shapely.wkt.loads(geom.ExportToWkt()) 
    
    if point.within(shape) and feature['MODE'] == mode:
        return True
    return False

In [5]:
def download_landsat_paths_and_rows():
    url = "https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip"
    r = urllib.request.urlopen(url)
    zip_file = zipfile.ZipFile(io.BytesIO(r.read()))
    zip_file.extractall("landsat-path-row")
    zip_file.close()

In [6]:
def get_paths_and_rows_parallel(coord_list, shapefile='landsat-path-row/WRS2_descending.shp',
                       mode='D', download_file=False):
    '''
    Get path, row for every coordinate in a list
    
    Inputs:
        coord_list: list of tuples (float, float). Expects (lon, lat) coordinates.
        shapefile: string, location of shapefile with landsat paths and rows
        mode: string, type of image to look for. Default is 'D', for daytime image
        download_file: whether to download landsat paths and rows shapefile
        
    Output:
        Tuple: Dictionary mapping each pair of coordinates to a (path, row) tuple,
            and set of all (path, row) tuples
    '''
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    
    if rank == 0 and download_file:
        download_landsat_paths_and_rows()
    
    wrs = ogr.Open(shapefile)
    layer = wrs.GetLayer(0)
    
    # evenly distribute list of coords across processes
    N = int(len(coord_list / size))
    start_ind = rank * N
    coord_list_slice = coord_list[start_ind: start_ind + N]
    
    feats_seen = []  # locations we've visited already
    coord_map = []
    for lon, lat in coord_list_slice:
        point = shapely.geometry.Point(lon, lat)
        found = False
        
        # first look through all the locations we've found already
        for feat in feats_seen:
            if checkPoint(feat, point, mode):
                found = True
                feature = feat
        
        # otherwise look through all the features in the world
        if not found:
            i = 0
            while not checkPoint(layer.GetFeature(i), point, mode):
                i += 1
            feature = layer.GetFeature(i)
            feats_seen.append(feature)
        
        coord_map.append( ((lon, lat), (feature['PATH'], feature['ROW'])) )
        
    # Gather all coordinates
    coord_map = comm.gather(coord_map, root=0)
        
    return coord_map

In [15]:
# source: https://stackoverflow.com/questions/44238525/how-to-iterate-over-files-in-an-s3-bucket
def iterate_bucket_items(bucket, prefix):
    """
    Generator that iterates over all objects in a given s3 bucket

    See http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.list_objects_v2 
    for return data format
    :param bucket: name of s3 bucket
    :return: dict of metadata for an object
    """
    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

    for page in page_iterator:
        if page['KeyCount'] > 0:
            for item in page['Contents']:
                yield item

In [17]:
t = 's3://landsat-pds/c1/L8/002/003'
for i in iterate_bucket_items(bucket=BUCKET,
                              prefix='s3://landsat-pds/c1/L8/002/003/'):
    print(i)

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [ ]:
def get_landsat_scene_strings(paths_rows_list):
    '''
    Construct list of strings corresponding to Landsat scenes
    to pull from S3 bucket
    '''
    base = 's3://landsat-pds/c1/L8/'
    
    scenes = []
    for path, row in paths_rows_list:
        # Pad path and row with zeroes as needed so they are each
        # 3 digits
        str_path = str(path).zfill(3)
        str_row = str(row).zfill(3)
        
        # s3 bucket with all Landsat scenes for given path and row
        scene_dir = base + str_path + '/' + str_row + '/'

In [7]:
coord_map, paths_rows = get_paths_and_rows(test)

In [8]:
paths_rows

{(7, 28), (141, 38), (186, 58), (188, 52)}

In [9]:
coord_map

{(-62.91692, 46.10125): (7, 28),
 (9.55137, 2.94979): (186, 58),
 (86.147, 31.66302): (141, 38),
 (9.28831, 12.13683): (188, 52)}